# Attribute Descriptors

The interpreter calls `__set_name__` on each descriptor it finds in a `class` body —— if the descriptor implements it. 

In this example the `LineItem` descriptor class doesn't need an `__init__`. Instead, `__set_name__` saves the same name of the storage attribute. 

In [1]:
class Quantity:

    # `self` is the descriptor instance (not the managed instance), 
    # `owner` is the managed class, and `name` is the name of the attribute of `owner` 
    # to which this descriptor instance was assigned in the class body of `owner`. 
    def __set_name__(self, owner, name):
        self.storage_name = name
    
    def __set__(self, instance, value):
        if value > 0:
            instance.__dict__[self.storage_name] = value
        else:
            msg = f'{self.storage_name} must be > 0'
            raise ValueError(msg)
    
    def __get__(self, instance, owner):
        return instance.__dict__[self.storage_name]

`__set__` is called when there is an attempt to assign to the managed attribute. Here, `self` is the descriptor instance (e.g., `LineItem.weight` or `LineItem.price`), `instance` is the managed instance (a `LineItem` instance), and the `value` is the value being assigned. 

Note that `__get__` receives three arguments: `self`, `instance` and `owner`. The `owner` argument is a reference to the managed class (e.g., `LineItem`). 

In [2]:
class LineItem:
    weight = Quantity()
    price = Quantity()

    def __init__(self, description, weight, price):
        self.descripton = description
        self.weight = weight
        self.price = price

    def subtotal(self):
        return self.weight * self.price

In [3]:
try:
    li = LineItem('as', -1, 0)
except ValueError:
    print('Opps')

Opps


In [4]:
li = LineItem("as", 1, 2)
li.subtotal()

2

### A New Descriptor Type

In [5]:
import abc

class Validated(abc.ABC):

    def __set_name__(self, owner, name):
        self.storage_name = name

    def __set__(self, instance, value):
        value = self.validated(self.storage_name, value)
        instance.__dict__[self.storage_name] = value

    @abc.abstractmethod
    def validated(self, name, value):
        """return validate value or raise ValueError"""

`__set__` delegates validation to the `validate` method, then uses the returned `value` to update the stored value. 

`validate` is an abstract method; this is the template method. 

## Overriding Versus Nonoverriding Descriptors

In [6]:
def cls_name(obj_or_cls):
    cls = type(obj_or_cls)
    if cls is type:
        cls = obj_or_cls
    return cls.__name__.split('.')[-1]

def display(obj):
    cls = type(obj)
    if cls is type:
        return f'<class {obj.__name__}>'
    elif cls in [type(None), int]:
        return repr(obj)
    else:
        return f'<{cls_name(obj)} object>'

def print_args(name, *args):
    pseudo_args = ', '.join(display(x) for x in args)
    print(f'-> {cls_name(args[0])}.__{name}__({pseudo_args})')

A descriptor that implements the `__set__` method is an *overriding descriptor*. 

An overriding descriptor can have no `__get__` method. 

A descriptor that does not implements the `__set__` method is an *nonoverriding descriptor*. 

In [7]:
class Overriding:
    """a.k.a. data descriptor or enforced descriptor"""

    def __get__(self, instance, owner):
        print_args('get', self, instance, owner) 

    def __set__(self, instance, value):
        print_args('set', self, instance, value)


class OverridingNoGet:  
    """an overriding descriptor without `__get__` """

    def __set__(self, instance, value):
        print_args('set', self, instance, value)


class NonOverriding: 
    """a.k.a. non-data or shadowable descriptor"""

    def __get__(self, instance, owner):
        print_args('get', self, instance, owner)


class Managed: 
    over = Overriding()
    over_no_get = OverridingNoGet()
    non_over = NonOverriding()

    def spam(self): 
        print(f'-> Managed.spam({display(self)})')

In [8]:
obj = Managed()  
obj.over
Managed.over
obj.over = 7
obj.over
obj.__dict__['over'] = 8

obj.over
print(vars(obj))

-> Overriding.__get__(<Overriding object>, <Managed object>, <class Managed>)
-> Overriding.__get__(<Overriding object>, None, <class Managed>)
-> Overriding.__set__(<Overriding object>, <Managed object>, 7)
-> Overriding.__get__(<Overriding object>, <Managed object>, <class Managed>)
-> Overriding.__get__(<Overriding object>, <Managed object>, <class Managed>)
{'over': 8}


In [9]:
obj.over_no_get
Managed.over_no_get
obj.over_no_get = 7
obj.over_no_get
obj.__dict__['over_no_get'] = 9
print(obj.over_no_get)
obj.over_no_get = 7
print(obj.over_no_get)

-> OverridingNoGet.__set__(<OverridingNoGet object>, <Managed object>, 7)
9
-> OverridingNoGet.__set__(<OverridingNoGet object>, <Managed object>, 7)
9


In [10]:
obj = Managed()
obj.non_over
obj.non_over = 7
print(obj.non_over)
Managed.non_over
del obj.non_over
obj.non_over

-> NonOverriding.__get__(<NonOverriding object>, <Managed object>, <class Managed>)
7
-> NonOverriding.__get__(<NonOverriding object>, None, <class Managed>)
-> NonOverriding.__get__(<NonOverriding object>, <Managed object>, <class Managed>)


### Methods Are Descriptors

In [12]:
obj = Managed()
print(obj.spam)
print(Managed.spam)
obj.spam = 7
print(obj.spam)

<bound method Managed.spam of <__main__.Managed object at 0x000001D23DE51090>>
<function Managed.spam at 0x000001D23DE78CA0>
7


In [14]:
import collections


class Text(collections.UserString):

    def __repr__(self):
        return 'Text({!r})'.format(self.data)

    def reverse(self):
        return self[::-1]

In [17]:
word = Text('forward')
print(word)

Text('forward')
print(word.reverse())

Text('drawrof')
print(Text.reverse(Text('backward')))

Text('drawkcab')
print(type(Text.reverse), type(word.reverse))

forward
drawrof
drawkcab
<class 'function'> <class 'method'>


A method called on the class works as a function. 

In [20]:
print(list(map(Text.reverse, ['repaid', (10, 20, 30), Text('stressed')])))

print(Text.reverse.__get__(word))
print(Text.reverse.__get__(None, Text))

['diaper', (30, 20, 10), Text('desserts')]
<bound method Text.reverse of Text('forward')>
<function Text.reverse at 0x000001D23DFE6320>


In [21]:
print(word.reverse)
print(word.reverse.__self__)
print(word.reverse.__func__ is Text.reverse)

<bound method Text.reverse of Text('forward')>
forward
True


The bound method object also has a `__call__` method, which handles the actual invocation. This method calls the original function referenced in `__func__`, passing the `__self__` attribute of the method as the first argument. 